In [3]:
import json

json_dir = "../results/detections/test_results.json"

with open(json_dir, "r") as f:
    data = json.load(f)
id_seen = set()
list_with_unique_id = []
for d in data:
    if d.get("image_id") in id_seen:
        continue
    id_seen.add(d.get("image_id"))
    list_with_unique_id.append(d)

In [4]:
def analyze_detection_results(results):
    """分析检测结果并返回各类问题的统计信息"""
    # 各类问题图像的ID列表
    no_detection_ids = []    # 未检测到任何物体的图像
    no_detection_id_gt = {}
    under_detection_ids = [] # 检测数量明显少于GT(少两个以上)的图像
    over_detection_ids = []  # 检测数量大于GT的图像
    
    # 统计数量
    total_images = len(results)
    total_gt = sum(len(r['ground_truth']) for r in results)
    total_detected = sum(len(r['detected']) for r in results)
    
    # 线路级别的检测结果
    line_stats = {}  # 每条线路的检测情况
    
    # 逐个分析每张图像
    for item in results:
        image_id = item['image_id']
        gt_count = len(item['ground_truth'])
        detected_count = len(item['detected'])
        
        # 统计各线路的检测情况
        gt_lines = [ann[0] for ann in item['ground_truth']]
        detected_lines = [int(ann[0]) if ann[0].isdigit() else ann[0] for ann in item['detected']]
        
        for line in gt_lines:
            if line not in line_stats:
                line_stats[line] = {'gt': 0, 'detected': 0, 'missed': 0}
            line_stats[line]['gt'] += 1
            if str(line) in [str(d) for d in detected_lines]:
                line_stats[line]['detected'] += 1
            else:
                line_stats[line]['missed'] += 1
        
        # 检查是否存在未检测到物体的情况
        if detected_count == 0 and gt_count > 0:
            no_detection_ids.append(image_id)
            no_detection_id_gt[image_id] = gt_lines
            
        # 检查是否检测数量明显少于GT
        if gt_count - detected_count >= 2:
            under_detection_ids.append(image_id)
            
        # 检查是否检测数量大于GT
        if detected_count > gt_count:
            over_detection_ids.append(image_id)
    
    # 计算整体准确率
    detection_rate = total_detected / total_gt if total_gt > 0 else 0
    
    return {
        'total_images': total_images,
        'total_gt': total_gt,
        'total_detected': total_detected,
        'detection_rate': detection_rate,
        'no_detection_ids': no_detection_ids,
        'under_detection_ids': under_detection_ids,
        'over_detection_ids': over_detection_ids,
        'line_stats': line_stats,
        'no_detection_id_gt': no_detection_id_gt
    }

def print_analysis_summary(analysis):
    """打印分析结果摘要"""
    print("==== 检测结果分析摘要 ====")
    print(f"总图像数: {analysis['total_images']}")
    print(f"总标注数: {analysis['total_gt']}")
    print(f"总检测数: {analysis['total_detected']}")
    print(f"检测率: {analysis['detection_rate']:.2%}")
    
    print("\n问题类型统计:")
    print(f"1. 未检测到任何物体的图像: {len(analysis['no_detection_ids'])}张")
    print(f"   图像ID: {analysis['no_detection_ids']}")
    
    print(f"\n2. 检测数量明显少于GT(少两个以上)的图像: {len(analysis['under_detection_ids'])}张")
    print(f"   图像ID: {analysis['under_detection_ids']}")
    
    print(f"\n3. 检测数量大于GT的图像: {len(analysis['over_detection_ids'])}张")
    print(f"   图像ID: {analysis['over_detection_ids']}")

    print(f"\n4. 未检测到物体的图像ID和对应的GT线路: {analysis['no_detection_id_gt']}")
    
    print("\n线路检测情况:")
    line_stats = analysis['line_stats']
    for line, stats in sorted(line_stats.items()):
        recall = stats['detected'] / stats['gt'] if stats['gt'] > 0 else 0
        print(f"线路 {line}: 总数 {stats['gt']}, 检出 {stats['detected']}, 漏检 {stats['missed']}, 召回率 {recall:.2%}")

def analyze_confusion_matrix(results):
    """分析线路间的混淆情况"""
    confusion = {} 
    false_positives = {} 
    misclassified = []  
    
    for item in results:
        image_id = item['image_id']
        gt_lines = {ann[0]: ann[1] for ann in item['ground_truth']}  
        
        for det in item['detected']:
            det_line_str = det[0] 
            det_line = int(det_line_str) if det_line_str.isdigit() else det_line_str
            det_box = det[1]  
            
            
            best_match = None
            best_iou = 0.3  
            
            for gt_line, gt_box in gt_lines.items():
                iou = calculate_iou(det_box, gt_box)
                if iou > best_iou:
                    best_iou = iou
                    best_match = gt_line
            
            # 记录混淆情况
            if best_match is not None:
                if best_match != det_line:
                    if best_match not in confusion:
                        confusion[best_match] = {}
                    if det_line not in confusion[best_match]:
                        confusion[best_match][det_line] = 0
                    confusion[best_match][det_line] += 1
                    
                    if image_id not in misclassified:
                        misclassified.append(image_id)
            else:
                if det_line not in false_positives:
                    false_positives[det_line] = 0
                false_positives[det_line] += 1
    
    return {
        'confusion': confusion,
        'false_positives': false_positives,
        'misclassified_ids': misclassified
    }

def calculate_iou(box1, box2):
    """计算两个边界框的IoU"""
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    x_left = max(x1_1, x1_2)
    y_top = max(y1_1, y1_2)
    x_right = min(x2_1, x2_2)
    y_bottom = min(y2_1, y2_2)
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
 
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
    

    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return iou


results = list_with_unique_id
analysis = analyze_detection_results(results)
print_analysis_summary(analysis)


confusion_analysis = analyze_confusion_matrix(results)
print("\n==== 线路混淆分析 ====")
#print("线路混淆情况:")
#for gt_line, preds in confusion_analysis['confusion'].items():
#    for pred_line, count in preds.items():
#        print(f"实际线路 {gt_line} 被误识别为线路 {pred_line}: {count}次")

print("\n误检情况:")
for line, count in confusion_analysis['false_positives'].items():
    print(f"线路 {line} 误检: {count}次")

print(f"\n存在分类错误的图像数: {len(confusion_analysis['misclassified_ids'])}")
print(f"图像ID: {confusion_analysis['misclassified_ids']}")



==== 检测结果分析摘要 ====
总图像数: 161
总标注数: 257
总检测数: 280
检测率: 108.95%

问题类型统计:
1. 未检测到任何物体的图像: 67张
   图像ID: [1, 2, 8, 16, 19, 20, 25, 26, 28, 32, 44, 46, 47, 49, 50, 56, 67, 70, 71, 82, 88, 89, 97, 103, 104, 106, 109, 110, 112, 116, 119, 121, 124, 130, 136, 142, 145, 146, 148, 157, 158, 160, 173, 175, 179, 181, 184, 187, 188, 190, 197, 200, 205, 206, 209, 217, 218, 229, 230, 233, 238, 242, 247, 248, 256, 259, 260]

2. 检测数量明显少于GT(少两个以上)的图像: 25张
   图像ID: [2, 8, 10, 16, 20, 28, 32, 40, 56, 67, 70, 71, 82, 88, 89, 97, 104, 119, 146, 200, 205, 206, 238, 242, 260]

3. 检测数量大于GT的图像: 52张
   图像ID: [4, 5, 13, 22, 23, 58, 59, 61, 65, 68, 73, 74, 79, 80, 85, 86, 100, 101, 131, 133, 134, 137, 139, 140, 151, 154, 155, 176, 178, 182, 185, 191, 193, 194, 196, 199, 202, 208, 212, 214, 215, 220, 223, 226, 232, 241, 244, 250, 251, 253, 254, 257]

4. 未检测到物体的图像ID和对应的GT线路: {1: [12], 2: [4, 4], 8: [1, 4, 7, 11, 14], 16: [1, 4, 14, 7, 11], 19: [4], 20: [1, 14, 7, 11], 25: [4], 26: [4], 28: [6, 12, 13], 32: [13, 12, 6]